In [1]:
import logging
import pickle
import numpy as np
from openmmtools.integrators import PeriodicNonequilibriumIntegrator
from simtk import unit, openmm
import argparse
import os
import time
import mdtraj as md
from tqdm import tqdm
from openmmtools.constants import kB

# Set up logger
_logger = logging.getLogger()
_logger.setLevel(logging.INFO)



In [2]:
outdir = "/data/chodera/zhangi/perses_benchmark/neq/14/38/"
phase = 'complex'
sim_number = 0
old_aa_name = 'lys'
new_aa_name = 'asn'
length = 1
cache = 1
cache_length = 1

## Run neq integrator

In [3]:
# Define lambda functions
x = 'lambda'
DEFAULT_ALCHEMICAL_FUNCTIONS = {
                             'lambda_sterics_core': x,
                             'lambda_electrostatics_core': x,
                             'lambda_sterics_insert': f"select(step({x} - 0.5), 1.0, 2.0 * {x})",
                             'lambda_sterics_delete': f"select(step({x} - 0.5), 2.0 * ({x} - 0.5), 0.0)",
                             'lambda_electrostatics_insert': f"select(step({x} - 0.5), 2.0 * ({x} - 0.5), 0.0)",
                             'lambda_electrostatics_delete': f"select(step({x} - 0.5), 1.0, 2.0 * {x})",
                             'lambda_bonds': x,
                             'lambda_angles': x,
                             'lambda_torsions': x}

# Define simulation parameters
# nsteps_eq = 25000 # 100 ps 
nsteps_eq = 1
nsteps_neq = 100
neq_splitting='V R H O R V'
timestep = 4.0 * unit.femtosecond
platform_name = 'CUDA'
cache_length = 1
temperature = 298.0 * unit.kelvin
kT = kB * temperature


In [4]:
# Read in vanilla htf
i = os.path.basename(os.path.dirname(outdir))
with open(os.path.join("/data/chodera/zhangi/perses_benchmark/neq/15/38/", f"{i}_{phase}.pickle"), 'rb') as f:
    htf = pickle.load(f)




In [5]:
# Read in lambda = 0 cache
with open(os.path.join(outdir, f"{i}_{phase}_{old_aa_name}_{cache_length}ns_snapshots.npy"), 'rb') as f:
    subset_pos = np.load(f)
positions = subset_pos[sim_number]
system = htf.hybrid_system


In [6]:
# Set up integrator
integrator = PeriodicNonequilibriumIntegrator(DEFAULT_ALCHEMICAL_FUNCTIONS, nsteps_eq, nsteps_neq, neq_splitting, timestep=timestep, temperature=temperature)

# Set up context
platform = openmm.Platform.getPlatformByName(platform_name)
if platform_name in ['CUDA', 'OpenCL']:
    platform.setPropertyDefaultValue('Precision', 'mixed')
if platform_name in ['CUDA']:
    platform.setPropertyDefaultValue('DeterministicForces', 'true')
context = openmm.Context(system, integrator, platform)
context.setPeriodicBoxVectors(*system.getDefaultPeriodicBoxVectors())
context.setPositions(positions)
context.setVelocitiesToTemperature(temperature)

# Minimize
openmm.LocalEnergyMinimizer.minimize(context)



In [7]:
# Run eq forward (0 -> 1)
forward_eq_old, forward_eq_new = list(), list()
for fwd_step in range(nsteps_eq):
    initial_time = time.time()
    _logger.info(f'before eq step: {fwd_step}, potential energy: {context.getState(getEnergy=True).getPotentialEnergy()}')
    integrator.step(1)
    _logger.info(f'after step: {fwd_step}, potential energy: {context.getState(getEnergy=True).getPotentialEnergy()}')
    elapsed_time = (time.time() - initial_time) * unit.seconds
    # if fwd_step % 2500 == 0:
    #     _logger.info(f'forward EQ step: {fwd_step}, took: {elapsed_time / unit.seconds} seconds')
    #     pos = context.getState(getPositions=True, enforcePeriodicBox=False).getPositions(asNumpy=True)
    #     old_pos = np.asarray(htf.old_positions(pos))
    #     new_pos = np.asarray(htf.new_positions(pos))
    #     forward_eq_old.append(old_pos)
    #     forward_eq_new.append(new_pos)

# Run neq forward (0 -> 1)
forward_works_master = list()
forward_neq_old, forward_neq_new = list(), list()
forward_works = [integrator.get_protocol_work(dimensionless=True)]
for fwd_step in range(nsteps_neq):
    initial_time = time.time()
    _logger.info(f'forward NEQ step: {fwd_step}, potential energy: {context.getState(getEnergy=True).getPotentialEnergy()}')
    integrator.step(1)
    elapsed_time = (time.time() - initial_time) * unit.seconds
    forward_works.append(integrator.get_protocol_work(dimensionless=True))
    if fwd_step % 2500 == 0:
        _logger.info(f'forward NEQ step: {fwd_step}, took: {elapsed_time / unit.seconds} seconds')
#         pos = context.getState(getPositions=True, enforcePeriodicBox=False).getPositions(asNumpy=True)
#         old_pos = np.asarray(htf.old_positions(pos))
#         old_traj = md.Trajectory(old_pos, md.Topology.from_openmm(htf._topology_proposal.old_topology))
#         old_pos_solute = old_traj.atom_slice(old_traj.top.select("not water")).xyz[0]
        
#         new_pos = np.asarray(htf.new_positions(pos))
#         new_traj = md.Trajectory(new_pos, md.Topology.from_openmm(htf._topology_proposal.new_topology))
#         new_pos_solute = new_traj.atom_slice(new_traj.top.select("not water")).xyz[0]
        
#         forward_neq_old.append(old_pos_solute)
#         forward_neq_new.append(new_pos_solute)
forward_works_master.append(forward_works)



INFO:root:before eq step: 0, potential energy: -3010836.3482994554 kJ/mol
INFO:root:after step: 0, potential energy: -2972137.435471569 kJ/mol
INFO:root:forward NEQ step: 0, potential energy: -2972137.435471569 kJ/mol
INFO:root:forward NEQ step: 0, took: 0.016508817672729492 seconds
INFO:root:forward NEQ step: 1, potential energy: -2923584.8821848957 kJ/mol
INFO:root:forward NEQ step: 2, potential energy: -2856595.285657404 kJ/mol
INFO:root:forward NEQ step: 3, potential energy: -2809212.915562595 kJ/mol
INFO:root:forward NEQ step: 4, potential energy: -2771765.027103324 kJ/mol
INFO:root:forward NEQ step: 5, potential energy: -2754686.353143754 kJ/mol
INFO:root:forward NEQ step: 6, potential energy: -2750355.2187641435 kJ/mol
INFO:root:forward NEQ step: 7, potential energy: -2755271.819716677 kJ/mol
INFO:root:forward NEQ step: 8, potential energy: -2761535.9008032745 kJ/mol
INFO:root:forward NEQ step: 9, potential energy: -2765362.5275417427 kJ/mol
INFO:root:forward NEQ step: 10, poten

In [9]:
# Read in lambda = 1 cache, if necessary
with open(os.path.join(outdir, f"{i}_{phase}_{new_aa_name}_{cache_length}ns_snapshots.npy"), 'rb') as f:
    subset_pos = np.load(f)
positions = subset_pos[sim_number]
context.setPositions(positions)




In [11]:
# Run eq reverse (1 -> 0)
reverse_eq_old, reverse_eq_new = list(), list()
for rev_step in range(nsteps_eq):
    initial_time = time.time()
    integrator.step(1)
    elapsed_time = (time.time() - initial_time) * unit.seconds
    # if rev_step % 2500 == 0:
    #     _logger.info(f'reverse EQ step: {rev_step}, took: {elapsed_time / unit.seconds} seconds')
    #     pos = context.getState(getPositions=True, enforcePeriodicBox=False).getPositions(asNumpy=True)
    #     old_pos = np.asarray(htf.old_positions(pos))
    #     new_pos = np.asarray(htf.new_positions(pos))
    #     reverse_eq_old.append(old_pos)
    #     reverse_eq_new.append(new_pos)

# Run neq reverse (1 -> 0)
reverse_works_master = list()
reverse_neq_old, reverse_neq_new = list(), list()
reverse_works = [integrator.get_protocol_work(dimensionless=True)]
for rev_step in range(nsteps_neq):
    initial_time = time.time()
    integrator.step(1)
    elapsed_time = (time.time() - initial_time) * unit.seconds
    reverse_works.append(integrator.get_protocol_work(dimensionless=True))
#     if rev_step % 2500 == 0:
    _logger.info(f'reverse NEQ step: {rev_step}, potential energy: {context.getState(getEnergy=True).getPotentialEnergy()}')
    pos = context.getState(getPositions=True, enforcePeriodicBox=False).getPositions(asNumpy=True)
#         old_pos = np.asarray(htf.old_positions(pos))
#         old_traj = md.Trajectory(old_pos, md.Topology.from_openmm(htf._topology_proposal.old_topology))
#         old_pos_solute = old_traj.atom_slice(old_traj.top.select("not water")).xyz[0]
        
    new_pos = np.asarray(htf.new_positions(pos))
    reverse_neq_new.append(new_pos)
#         new_traj = md.Trajectory(new_pos, md.Topology.from_openmm(htf._topology_proposal.new_topology))
#         new_pos_solute = new_traj.atom_slice(new_traj.top.select("not water")).xyz[0]

#         reverse_neq_old.append(old_pos_solute)
#         reverse_neq_new.append(new_pos_solute)
reverse_works_master.append(reverse_works)



INFO:root:reverse NEQ step: 0, potential energy: -2486817.538300059 kJ/mol
INFO:root:reverse NEQ step: 1, potential energy: -2502310.322995426 kJ/mol
INFO:root:reverse NEQ step: 2, potential energy: -2509552.33700009 kJ/mol
INFO:root:reverse NEQ step: 3, potential energy: -2510232.5263973093 kJ/mol
INFO:root:reverse NEQ step: 4, potential energy: -2507741.2093519596 kJ/mol
INFO:root:reverse NEQ step: 5, potential energy: -2505484.3187867883 kJ/mol
INFO:root:reverse NEQ step: 6, potential energy: -2503960.028456036 kJ/mol
INFO:root:reverse NEQ step: 7, potential energy: -2503441.8894458218 kJ/mol
INFO:root:reverse NEQ step: 8, potential energy: -2502362.852590573 kJ/mol


KeyboardInterrupt: 

In [ ]:
# Save works
with open(os.path.join(args.dir, f"{i}_{args.phase}_{args.sim_number}_forward.npy"), 'wb') as f:
    np.save(f, forward_works_master)
with open(os.path.join(args.dir, f"{i}_{args.phase}_{args.sim_number}_reverse.npy"), 'wb') as f:
    np.save(f, reverse_works_master)

# Save trajs
# with open(os.path.join(args.dir, f"{i}_{args.phase}_{args.sim_number}_forward_eq_old.npy"), 'wb') as f:
#     np.save(f, forward_eq_old)
# with open(os.path.join(args.dir, f"{i}_{args.phase}_{args.sim_number}_forward_eq_new.npy"), 'wb') as f:
#     np.save(f, forward_eq_new)
# with open(os.path.join(args.dir, f"{i}_{args.phase}_{args.sim_number}_reverse_eq_old.npy"), 'wb') as f:
#     np.save(f, reverse_eq_old)
# with open(os.path.join(args.dir, f"{i}_{args.phase}_{args.sim_number}_reverse_eq_new.npy"), 'wb') as f:
#     np.save(f, reverse_eq_new)

with open(os.path.join(args.dir, f"{i}_{args.phase}_{args.sim_number}_forward_neq_old.npy"), 'wb') as f:
    np.save(f, forward_neq_old)
with open(os.path.join(args.dir, f"{i}_{args.phase}_{args.sim_number}_forward_neq_new.npy"), 'wb') as f:
    np.save(f, forward_neq_new)
with open(os.path.join(args.dir, f"{i}_{args.phase}_{args.sim_number}_reverse_neq_old.npy"), 'wb') as f:
    np.save(f, reverse_neq_old)
with open(os.path.join(args.dir, f"{i}_{args.phase}_{args.sim_number}_reverse_neq_new.npy"), 'wb') as f:
    np.save(f, reverse_neq_new)
